In [ ]:
### 봉은사역 근처 top5 아파트 시세
# 미켈란 107, 풍림 1차, 아이파크 삼성, 삼부아그레빌, 래미안 삼성1차(301,302,303동)

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [3]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [4]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [5]:
## 미켈란 107
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=50496"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75636954134152550496']


In [ ]:
## 미켈란 107
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=50496&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)

key = '75636954134152550496'

# 가격과 전월세 컬럼 복호화
df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key))
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))

df_1['가격(숫자)'] = df_1['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_1

In [8]:
## 풍림 1차
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=1354"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7543695431597791354']


In [ ]:
## 풍림1차
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=1354&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '7543695431597791354'

# 가격과 전월세 컬럼 복호화
df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))

df_2['가격(숫자)'] = df_2['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_2

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.06,12,"18억 8,000",,1800008000
1,2025.03.22,19,"19억 3,000",,1900003000
2,2025.03.21,13,14억,,1400000000
3,2025.03.08,6,19억,,1900000000
4,2025.02.28,9,26억,,2600000000
...,...,...,...,...,...
193,2006.02.24,17,"3억 8,500",,300008500
194,2006.01.25,13,6억,,600000000
195,2006.01.17,11,"7억 3,500",,700003500
196,2006.01.17,8,"5억 4,500",,500004500


In [10]:
## 아이파크 삼성
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=53516"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75636954122978553516']


In [11]:
## 아이파크 삼성
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=53516&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '75636954122978553516'

# 가격과 전월세 컬럼 복호화
df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))

df_3['가격(숫자)'] = df_3['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_3

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.18,30,75억,,7500000000
1,2025.06.16,25,66억,,6600000000
2,2025.06.04,19,67억,,6700000000
3,2025.03.22,34,71억,,7100000000
4,2025.02.21,32,80억,,8000000000
...,...,...,...,...,...
266,2006.09.28,43,44억,,4400000000
267,2006.09.26,30,"36억 2,000",,3600002000
268,2006.09.01,19,32억,,3200000000
269,2006.07.18,33,27억,,2700000000


In [12]:
## 삼부아그레빌
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=50511"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75136954134097050511']


In [14]:
## 삼부아그레빌
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=50511&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '75136954134097050511'

# 가격과 전월세 컬럼 복호화
df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))

df_4['가격(숫자)'] = df_4['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_4

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.27,10,"22억 2,000",,2200002000
1,2025.06.26,10,"22억 2,000",,2200002000
2,2025.06.18,4,20억,,2000000000
3,2025.03.19,9,18억,,1800000000
4,2025.02.24,11,18억,,1800000000
5,2023.12.28,12,19억,,1900000000
6,2019.11.15,2,"16억 8,000",,1600008000
7,2019.07.18,10,"15억 5,000",,1500005000
8,2019.07.05,12,"15억 2,000",,1500002000
9,2018.03.24,13,"12억 5,000",,1200005000


In [15]:
## 래미안 삼성1차(301,302,303동)
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20040514"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75436880171085920040514']


In [16]:
## 래미안 삼성 1차(301,302,303동)
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20040514&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=50&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '75436880171085920040514'

# 가격과 전월세 컬럼 복호화
df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))

df_5['가격(숫자)'] = df_5['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_5

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.02,5,"33억 5,000",,3300005000
1,2025.03.16,7,"25억 3,500",,2500003500
2,2025.01.16,7,"21억 2,000",,2100002000
3,2024.07.31,7,"21억 2,000",,2100002000
4,2024.07.31,7,"21억 3,000",,2100003000
5,2023.08.07,3,"25억 8,000",,2500008000
6,2023.03.09,15,19억,,1900000000
7,2022.09.27,11,33억,,3300000000
8,2020.11.14,6,23억,,2300000000
9,2020.06.18,2,19억,,1900000000
